## Part 1

In [ ]:
#Import Libs
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import etree

In [ ]:
#Define the URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [ ]:
#Access the Data from Wiki and save as DataFrame
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
tables = soup.select('table[class = "wikitable sortable"]')
df_list = []
for table in tables:
    df_list.append(pd.concat(pd.read_html(table.prettify())))
    pass
df_wiki = pd.concat(df_list)
df_wiki.head()

In [ ]:
#Clean the DataFrame
df_data=df_wiki[~df_wiki['Borough'].isin(['Not assigned'])].reset_index().drop(columns='index')
df_data.head()

In [ ]:
#Print the number of rows of DataFrame
df_data.shape

## Part 2

In [ ]:
#Get Access to the Geo Data
geo_data_url = 'http://cocl.us/Geospatial_data'
df_geo_data = pd.read_csv(geo_data_url)
df_geo_data.head()

In [ ]:
#Combine the DataFrame
df_data_total = pd.merge(df_data,df_geo_data,how='inner',on='Postal Code')
df_data_total.head()

## Part 3

In [ ]:
#Visualize the map
!pip install folium
import folium

In [ ]:
#Visualize the Canada Map
map_canada = folium.Map(location=[43.8, -79.4], zoom_start=11)
map_canada

In [ ]:
#Mark on the MAP
incidents = folium.map.FeatureGroup()
for lat, lng, in zip(df_data_total.Latitude, df_data_total.Longitude):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
map_canada.add_child(incidents)

In [ ]:
#It seems like that neighborhoods distribution in Toronto is relatively even